In [2]:
#*************************************************************************
#1.Importamos la libreria NLTK
#*************************************************************************
import nltk


#*************************************************************************
#2.Creamos una texto de entrada a nuestra cadena NLP
#*************************************************************************
text = "I didn't notice my animals were uglier than yours! I'm sorry..."
print ("\n\n1. Texto:",text)


#*************************************************************************
#2.Dividimos el texto en frases
#*************************************************************************
sentences = nltk.tokenize.sent_tokenize(text)
print ("\n\n2. Frases:",sentences)


#*****************************************************************************
#3.Tokenización: tokenizamos el texto, es decir dividimos el texto en tokens
#*****************************************************************************
tokens = nltk.word_tokenize(text)
print ("\n\n3. Tokens:",tokens)



#*************************************************************************
#4.Análisis morfológico: asignamos una etiqueta morfologica a cada token
#*************************************************************************
tagged = nltk.pos_tag(tokens)
print ("\n\n4. Analisis Morfologico:",tagged)



#*******************************************************************  
#5.Stemming: obtenemos la raíz (en inglés 'stem') de cada token
#*******************************************************************  
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
print ("\n\n5. Stems: ")
for tok in tokens:
    print (stemmer.stem(tok.lower()))
  
    
 
 
#*******************************************************************  
#6.Lematización: obtenemos el lema de cada token 
#*******************************************************************  
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
#El lematizador de wordnet solo reconoce 4 etiquetas POS: a (adjetivo), r(adverbio),n (nombre),v(verbo). 
#Así que debemos hacer una conversión del formato Penn Tree Bank al formato wordnet (ej: NN->n, JJ->a, RB->r, VB->V, ...)
from nltk.corpus import wordnet
wnTags = {'N':wordnet.NOUN,'J':wordnet.ADJ,'V':wordnet.VERB,'R':wordnet.ADV} 
print ("\n\n\n6. Lemas: ")
for (tok,tag) in tagged:
    #wordnet no contiene las formas abreviadas 'm  y  n't así que las introducimos nosotros para que lematice bien
    if tok=='\'m':
        tok = 'am'
    if tok=='\'s':
        tok = 'is'
    if tok=='n\'t':
        tok = 'not'
    tag = tag[:1]
    lemma = lemmatizer.lemmatize(tok.lower(),wnTags.get(tag,wordnet.NOUN))
    #otra forma alternativa de obtener el lema hubiera sido llamar directamente a la funcion wordnet.morphy, que hace lo mismo:
    #lemma = wordnet.morphy(tok.lower(),wnTags.get(tag,wordnet.NOUN))
    if lemma is None: #Si wordnet no contiene la palabra, supondremos que el lema es igual al token
       lemma = tok.lower() 
    print (lemma)

    
    
    
    
#*******************************************************************    
#7.Análisis sintáctico
#******************************************************************* 

#Partimos de una frase de un conocido texto de Groucho Marx, con una clara ambigüedad: 
#"While hunting in Africa, I shot an elephant in my pijamas. How he got into my pijamas, I don't know."
#¿Groucho estaba en pijama o el elefante estaba dentro de su pijama?
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pijamas']

#Creamos nuestra propia Gramatica Libre de Contexto (en inglés CFG)
grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pijamas'
V -> 'shot' | 'did'
P -> 'in'
""")


#Generamos un parser sintáctico capaz de reconocer la gramática
parser = nltk.ChartParser(grammar, trace=1)
print ('\n\n\n7. Analisis sintactico:\n')
for tree in parser.parse(sent):
    print(tree,'\n')
    tree.draw()
nltk.parse.chart.demo(2, print_times=False, trace=1, sent='I saw a dog', numparses=1)




1. Texto: I didn't notice my animals were uglier than yours! I'm sorry...


2. Frases: ["I didn't notice my animals were uglier than yours!", "I'm sorry..."]


3. Tokens: ['I', 'did', "n't", 'notice', 'my', 'animals', 'were', 'uglier', 'than', 'yours', '!', 'I', "'m", 'sorry', '...']


4. Analisis Morfologico: [('I', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('notice', 'VB'), ('my', 'PRP$'), ('animals', 'NNS'), ('were', 'VBD'), ('uglier', 'JJR'), ('than', 'IN'), ('yours', 'JJR'), ('!', '.'), ('I', 'PRP'), ("'m", 'VBP'), ('sorry', 'JJ'), ('...', ':')]


5. Stems: 
i
did
n't
notic
my
anim
were
uglier
than
your
!
i
'm
sorri
...



6. Lemas: 
i
do
not
notice
my
animal
be
ugly
than
yours
!
i
be
sorry
...



7. Analisis sintactico:

|.  I  . shot.  an .eleph.  in .  my .pijam.|
|[-----]     .     .     .     .     .     .| [0:1] 'I'
|.     [-----]     .     .     .     .     .| [1:2] 'shot'
|.     .     [-----]     .     .     .     .| [2:3] 'an'
|.     .     .     [-----]     .     .     .| 